# Evaluation of RAG architectures

In [ ]:
from llms.rag.faiss import FAISS
from llms.rag.coala import CoALA
from llms.agents.react import ReActAgent
from llms.clients.gpt import GPTClient
from llms.settings import settings
from llms.rag.tools import Tools
from llms.rag.faiss import DistanceMetric
from llms.evaluation.code import evaluate_code_generation, ConfigGrid, RAG, RAGRetriever

In [ ]:
gpt_4_client = GPTClient(
    client_id=settings.CLIENT_ID,
    client_secret=settings.CLIENT_SECRET,
    auth_url=settings.AUTH_URL,
    api_base=settings.API_BASE,
    deployment_id='gpt-4-32k',
    max_response_tokens=100,
    temperature=0.0,
)

gpt_35_client = GPTClient(
    client_id=settings.CLIENT_ID,
    client_secret=settings.CLIENT_SECRET,
    auth_url=settings.AUTH_URL,
    api_base=settings.API_BASE,
    deployment_id='gpt-35-turbo-16k',
    max_response_tokens=100,
    temperature=0.0,
)

In [ ]:
print(settings.STANDARD_SYSTEM_INSTRUCTION)

In [ ]:
config = ConfigGrid(
    llms=[gpt_4_client, gpt_35_client],
    rag=RAG(
        retrievers=[RAGRetriever.NONE, RAGRetriever.RAG, RAGRetriever.CoALA, RAGRetriever.RAG_AS_TOOL, RAGRetriever.CoALA_AS_TOOL],
        distance_metrics=[DistanceMetric.EUCLIDEAN_DISTANCE, DistanceMetric.MAX_INNER_PRODUCT, DistanceMetric.COSINE_SIMILARITY],
        num_search_results=[1, 3, 5],
        similarity_search_score_thresholds=[0.0, 0.5, 0.75],
        texts=[],
        text_chunk_sizes=[128, 256, 512],
        use_weighted_average_of_text_chunks=[True, False],
    )
)

In [ ]:
evaluate_code_generation(config, [])